# Setup

In [19]:
import pymupdf4llm
import pathlib
from tqdm import tqdm
from langchain.schema import Document
import retrieval_utils
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

import importlib
importlib.reload(retrieval_utils)

<module 'retrieval_utils' from 'c:\\Users\\ankum\\OneDrive\\Desktop\\vivado_rag\\retrieval_utils.py'>

# Retrieving Chunks

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization = True)
print("Total chunks in FAISS index:", vectorstore.index.ntotal)

C:\Users\ankum\AppData\Local\Temp\ipykernel_12704\3598431607.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Total chunks in FAISS index: 1119


In [21]:
question_page_pairs = retrieval_utils.load_questions_from_csv("vivado_rag_question_bank.csv")
print(f"Loaded {len(question_page_pairs)} questions")

Loaded 10 questions


In [22]:
[qpp[0] for qpp in question_page_pairs]

['How can I add enumerations using TCL commands?',
 'What key settings can I use when generating an encrypted bitstream?',
 'I need to both watch a signal’s value in the Waveform window and use that same signal to kick off a capture. What probe type should I pick?',
 'How can I create a custom debug probe on an ILA core in the Vivado Hardware Manager?',
 'I need to both validate and optimize my FPGA’s high-speed serial links in-system and, at the same time, issue AXI reads and writes over JTAG without a processor.',
 'Can I get some information on how to do serial I/O hardware debugging in Vivado?',
 'I’d like to script an IBERT eye-scan on my KC705 board using Vivado’s Tcl interface',
 'When I enable Auto Re-trigger on an ILA core, I notice I sometimes miss signal cycles between captures. How can I fix that?',
 'How do I add probes from my ILA core into the Waveform window?',
 'How can I read on-chip temperature data at run time?']

# Retrieval Tests

In [29]:
# k = 5 gives 85% recall
for question, actual_pages in question_page_pairs:
    docs = vectorstore.similarity_search(question, k = 5)
    # extract their page numbers
    retrieved_pages = [doc.metadata["page_number"] for doc in docs]
    print("Question:", question)
    print("Retrieved pages:", list(set(retrieved_pages)))
    print("Actual pages:   ", actual_pages)
    print("-" * 40)

Question: How can I add enumerations using TCL commands?
Retrieved pages: [308, 309, 245, 214, 215]
Actual pages:    [214]
----------------------------------------
Question: What key settings can I use when generating an encrypted bitstream?
Retrieved pages: [80, 81, 82]
Actual pages:    [82]
----------------------------------------
Question: I need to both watch a signal’s value in the Waveform window and use that same signal to kick off a capture. What probe type should I pick?
Retrieved pages: [259, 142, 211, 185, 190]
Actual pages:    [142]
----------------------------------------
Question: How can I create a custom debug probe on an ILA core in the Vivado Hardware Manager?
Retrieved pages: [184, 131, 180, 183]
Actual pages:    [184]
----------------------------------------
Question: I need to both validate and optimize my FPGA’s high-speed serial links in-system and, at the same time, issue AXI reads and writes over JTAG without a processor.
Retrieved pages: [11, 147, 237, 238]
Ac

In [30]:
# k = 10 gives 85% recall also
for question, actual_pages in question_page_pairs:
    docs = vectorstore.similarity_search(question, k = 10)
    # extract their page numbers
    retrieved_pages = [doc.metadata["page_number"] for doc in docs]
    print("Question:", question)
    print("Retrieved pages:", list(set(retrieved_pages)))
    print("Actual pages:   ", actual_pages)
    print("-" * 40)

Question: How can I add enumerations using TCL commands?
Retrieved pages: [210, 242, 308, 309, 245, 214, 246, 180, 215, 219]
Actual pages:    [214]
----------------------------------------
Question: What key settings can I use when generating an encrypted bitstream?
Retrieved pages: [77, 79, 80, 81, 82, 83, 24]
Actual pages:    [82]
----------------------------------------
Question: I need to both watch a signal’s value in the Waveform window and use that same signal to kick off a capture. What probe type should I pick?
Retrieved pages: [259, 260, 198, 142, 211, 183, 185, 190]
Actual pages:    [142]
----------------------------------------
Question: How can I create a custom debug probe on an ILA core in the Vivado Hardware Manager?
Retrieved pages: [131, 165, 166, 179, 180, 277, 183, 184]
Actual pages:    [184]
----------------------------------------
Question: I need to both validate and optimize my FPGA’s high-speed serial links in-system and, at the same time, issue AXI reads and w

In [35]:
# k = 12 gives 90% recall also
for question, actual_pages in question_page_pairs:
    docs = vectorstore.similarity_search(question, k = 12)
    # extract their page numbers
    retrieved_pages = [doc.metadata["page_number"] for doc in docs]
    print("Question:", question)
    print("Retrieved pages:", list(set(retrieved_pages)))
    print("Actual pages:   ", actual_pages)
    print("-" * 40)

Question: How can I add enumerations using TCL commands?
Retrieved pages: [211, 210, 242, 308, 309, 245, 214, 246, 180, 215, 219, 244]
Actual pages:    [214]
----------------------------------------
Question: What key settings can I use when generating an encrypted bitstream?
Retrieved pages: [77, 79, 80, 81, 82, 83, 24, 27]
Actual pages:    [82]
----------------------------------------
Question: I need to both watch a signal’s value in the Waveform window and use that same signal to kick off a capture. What probe type should I pick?
Retrieved pages: [259, 260, 198, 142, 211, 183, 185, 190]
Actual pages:    [142]
----------------------------------------
Question: How can I create a custom debug probe on an ILA core in the Vivado Hardware Manager?
Retrieved pages: [131, 165, 166, 12, 179, 180, 277, 183, 184, 155]
Actual pages:    [184]
----------------------------------------
Question: I need to both validate and optimize my FPGA’s high-speed serial links in-system and, at the same time